In [ ]:
#  functions
import pypsa, pandas as pd, numpy as np
print("PyPSA:", pypsa.__version__)
def summarize(n):
    tables = ["buses","loads","generators","lines","links","transformers","storage_units"]
    counts = {t: getattr(n, t).shape[0] for t in tables}
    print(f"Network: {getattr(n, 'name', 'unknown')}")
    print("Snapshots:", len(getattr(n, "snapshots", [])))
    for t,c in counts.items():
        print(f"{t:14s}: {c}")
    if hasattr(n, "carriers") and not n.carriers.empty:
        print("\nCarriers defined:", ", ".join(n.carriers.index))
    else:
        print("\nCarriers defined: none")

In [99]:
n = pypsa.Network()
n.name = "toy-24h"
# time snapshots, add number of shots, define yearly scale
hours = pd.date_range("2025-01-01", periods=24, freq="h")
n.set_snapshots(hours)
H=len(n.snapshots)
w = 8760/H

# define carriers
n.add("Carrier","electricity")
n.add("Carrier","wind")
n.add("Carrier","gas")
n.add("Carrier","battery")  
n.add("Carrier", "coal")
n.add("Carrier", "hydro")
n.add("Carrier", "shed")
# add bus
n.add("Bus","bus0", carrier="electricity")

# add load, match snapshots, assume sin pattern, vary 5 MW to 15 MW throughout day
hangles = np.linspace(0, 2*np.pi, H, endpoint=False)
p = 10 + 5*np.sin(hangles)
n.add("Load","demand", bus="bus0",
      p_set=p)

# add wind generator, small random marginal cost, random loads between 0 and 1, scale CAPEX to daily
rng = np.random.default_rng(42)
n.add("Generator","wind", bus="bus0", carrier="wind",
      p_max_pu=rng.random(H),
      p_nom_extendable=True,
      marginal_cost=2,
      capital_cost=900)

# add gas generator, built for backup, high marginal cost
n.add("Generator","ocgt", bus="bus0", carrier="gas",
      p_nom_extendable=True,
      marginal_cost=70,
      capital_cost=500)

#  add battery storage, high efficiency
n.add("StorageUnit","battery", bus="bus0",
      p_nom_extendable=True, max_hours=2,
      efficiency_store=0.95, efficiency_dispatch=0.95,
      carrier="battery",
      capital_cost=120)
#add coal power plant (high capital cost, lower marginal than gas, stable load, include variation)
n.add(
    "Generator", "coal_pp",
    bus="bus0", carrier="coal",
    p_nom_extendable=True,
    capital_cost=650,
    marginal_cost=45.0,
    p_min_pu=0.50,
    ramp_limit_up=0.10,
    ramp_limit_down=0.10
)
# add hydro reservoir, high capital cost, no fuel cost
n.add("StorageUnit", "hydro_dam", bus="bus0", carrier="hydro",
      p_nom_extendable=True,
      max_hours=90,     #90 hour reservoir
      efficiency_store=1.0,
      efficiency_dispatch=0.90, #loss during conversion
      standing_loss=0.001,         # evaporation loss
      capital_cost=1000,
      marginal_cost=0.0)

#model inflow of dam
inflow = pd.Series(5.0 + 3.0*np.sin(hangles),
                   index=n.snapshots)

# assign the inflow to the dam
n.storage_units_t.inflow.loc[:, "hydro_dam"] = inflow

#Add high-cost shed generator
VOLL = 5000.0  # placeholder high cost, releiabilty tester
n.add("Generator", "shed", bus="bus0", carrier="shed",
      p_nom_extendable=True, marginal_cost=VOLL, capital_cost=0.0)

# toy emission dictionary
CO2_INT = {"gas": 0.40, "coal": 0.90}  # emission only for gal and coal, constant

#define kpi function
def kpis(n):
    em = 0.0 #define the accumulator
    for g, row in n.generators.iterrows(): #loop over the generators
        em += CO2_INT.get(row.carrier, 0.0) * n.generators_t.p[g].sum() #add emissions based on number of generators (intensity*energy)

    # check for unment demand
    shedt = n.generators_t.p["shed"].sum() if "shed" in n.generators_t.p.columns else 0.0

    #capacity mix (MW)
    mix = n.generators.copy() #copy geneator list
    cap_col = "p_nom_opt" if "p_nom_opt" in mix.columns else "p_nom" #group the p_nom
    mix["cap_MW"] = mix[cap_col]#add a new column of p_nom
    caps = mix.groupby("carrier")["cap_MW"].sum().to_dict() #collapse the common columns and sum by carrier

    #average prices for both snapshots and buses
    mp = float(n.buses_t.marginal_price.mean().mean())

  #Curtailment by carrier (avaialable energy not produced)
    curtailment_by_carrier = {} #start with empty dict
    if hasattr(n.generators_t, "p_max_pu"): #check for generators of variable output "p_max_pu"
        for g in n.generators.index: #search for generator in dict
            if g in n.generators_t.p.columns and g in n.generators_t.p_max_pu.columns: #check that generators has both dispatch time series and avaialbility series
                p_nom_g = n.generators.at[g, cap_col] #get the generator's capcaity
                available = n.generators_t.p_max_pu[g] * p_nom_g #multiply availabilty by capacity
                curtailed = (available - n.generators_t.p[g]).clip(lower=0).sum() #calculate curtailment and sum
                if curtailed > 0: #check if curtailment occured
                    carr = n.generators.at[g, "carrier"]# isolate curtailed generator
                    curtailment_by_carrier[carr] = curtailment_by_carrier.get(carr, 0.0) + float(curtailed) #poupulat the dict with curtailed carrties and values

# neat output of resulting kpis
    out = dict(system_cost=float(n.objective),
               emissions_t=float(em),
               unserved_MWh=float(shed),
               mean_price_eur_per_MWh=mp,
               **caps)

    for carr, val in curtailment_by_carrier.items(): #include the curtailemnt dictionary
        out[f"curtail_{carr}_MWh"] = val

    return out


# optimize free variables
n.optimize(solver_name="highs")

#print energy balance checker
balance = (n.generators_t.p.sum(axis=1)
           + n.storage_units_t.p_dispatch.sum(axis=1)
           - n.storage_units_t.p_store.sum(axis=1)
           - n.loads_t.p.sum(axis=1)).round(6)
print("\nPower balance residual (should be 0 each hour):")
print(balance.head(9))

# first 9 hours for all storage units
print("\nStorage discharge p_dispatch (MW), first 9h:")
print(n.storage_units_t.p_dispatch.head(9))
print("\nStorage charge p_store (MW), first 9h:")
print(n.storage_units_t.p_store.head(9))
print("\nStorage state of charge (MWh), first 9h:")
print(n.storage_units_t.state_of_charge.head(9))

#print size and dispatch
print("Optimal capacities (MW):\n", n.generators.p_nom_opt)
print("\nDispatch (first 9 hours, MW):\n", n.generators_t.p.head(9))

#print hydro inflow
if "hydro_dam" in n.storage_units.index:
    print("\n[hydro_dam] inflow (MW), first 9h:")
    print(n.storage_units_t.inflow["hydro_dam"].head(9))


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.06s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 270 primals, 604 duals
Objective: 1.73e+04
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Generator-ext-p-ramp_limit_up, Generator-ext-p-ramp_limit_down, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-mg_x_rsc has 604 rows; 270 cols; 1294 nonzeros
Coefficient ranges:
  Matrix [6e-02, 9e+01]
  Cost   [2e+00, 5e+03]
  Bound  [2e+00, 8e+00]
  RHS    [2e+00, 2e+01]
Presolving model
358 rows, 269 cols, 1024 nonzeros  0s
Dependent equations search running on 71 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
357 rows, 268 cols, 1022 nonzeros  0s
Presolve : Reductions: rows 357(-247); columns 268(-2); elements 1022(-272)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0800000000e+06 Ph1: 72(75976); Du: 24(1080) 0s
        229     1.7297279097e+04 Pr: 0(0); Du: 0(1.42109e-14) 0s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-mg_x_rsc
Model status        : Optimal
Simplex